# 🏠 Notebook 5 : Application Gradio
## Ames Housing - Interface de Prédiction des Prix Immobiliers

---

### 🎯 Objectif
Créer une interface utilisateur interactive pour prédire les prix des maisons en utilisant le modèle ML entraîné.

### 📋 Fonctionnalités
- Formulaire de saisie pour les caractéristiques d'une maison
- Prédiction instantanée du prix
- Visualisation des comparaisons
- Upload de fichier CSV pour prédictions multiples

---
## 1. Installation et Configuration

In [1]:
# Installation de Gradio si nécessaire
!pip install gradio -q

import gradio as gr
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

print('✅ Gradio chargé!')


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
✅ Gradio chargé!


In [2]:
# Charger le modèle et les préprocesseurs
with open('../models/final_model.pkl', 'rb') as f:
    model_package = pickle.load(f)
    model = model_package['model']
    feature_names = model_package['feature_names']

with open('../models/preprocessors.pkl', 'rb') as f:
    preprocessors = pickle.load(f)
    scaler = preprocessors['scaler']

print(f'✅ Modèle chargé: {model_package["model_name"]}')
print(f'📊 R² Test: {model_package["metrics"]["r2"]:.4f}')
print(f'📊 RMSE Test: ${model_package["metrics"]["rmse"]:,.2f}')

✅ Modèle chargé: Extra Trees
📊 R² Test: 0.9142
📊 RMSE Test: $21,773.01


---
## 2. Fonction de Prédiction

In [4]:
# Charger les données prétraitées pour baseline et les features skewed
df_prep = pd.read_csv('../data/data_preprocessed.csv')
median_baseline = df_prep[feature_names].median()
highly_skewed = preprocessors['highly_skewed_features']

def predict_price(
    overall_qual, gr_liv_area, garage_cars, garage_area, 
    total_bsmt_sf, first_flr_sf, year_built, full_bath,
    totrms_abvgrd, fireplaces
):
    """
    Prédit le prix d'une maison en utilisant le modèle ML entraîné.
    Applique log1p aux features skewed avant le scaling.
    """
    try:
        # Utiliser la baseline médiane (valeurs déjà scaled)
        input_data = pd.DataFrame([median_baseline], columns=feature_names)
        
        # Calculer les features dérivées
        second_flr_sf = max(0, gr_liv_area - first_flr_sf)
        total_sf = gr_liv_area + total_bsmt_sf
        total_bathrooms = full_bath + 0.5
        house_age = 2010 - year_built
        quality_area = overall_qual * gr_liv_area
        
        # Features à transformer
        user_features = {
            'OverallQual': overall_qual,
            'GrLivArea': gr_liv_area,
            'TotalBsmtSF': total_bsmt_sf,
            '1stFlrSF': first_flr_sf,
            '2ndFlrSF': second_flr_sf,
            'GarageCars': garage_cars,
            'GarageArea': garage_area,
            'YearBuilt': year_built,
            'YearRemodAdd': year_built,
            'FullBath': full_bath,
            'TotRmsAbvGrd': totrms_abvgrd,
            'Fireplaces': fireplaces,
            'TotalSF': total_sf,
            'TotalBathrooms': total_bathrooms,
            'HouseAge': house_age,
            'RemodAge': house_age,
            'QualityArea': quality_area,
            'HasGarage': 1 if garage_cars > 0 else 0,
            'HasBasement': 1 if total_bsmt_sf > 0 else 0,
            'HasFireplace': 1 if fireplaces > 0 else 0,
        }
        
        # Transformer et mettre à jour les features
        scaler_features = list(scaler.feature_names_in_)
        for feat, val in user_features.items():
            if feat in feature_names and feat in scaler_features:
                # Appliquer log1p si feature skewed
                transformed_val = np.log1p(val) if feat in highly_skewed else val
                # Scaler
                feat_idx = scaler_features.index(feat)
                scaled_val = (transformed_val - scaler.mean_[feat_idx]) / scaler.scale_[feat_idx]
                input_data[feat] = scaled_val
        
        # Prédiction
        prediction = model.predict(input_data)[0]
        prediction = max(50000, min(prediction, 800000))
        
        result = f'💰 Prix estimé: ${prediction:,.0f}'
        
        if prediction < 150000:
            interpretation = '🏠 Maison économique'
        elif prediction < 250000:
            interpretation = '🏡 Maison moyenne'
        elif prediction < 400000:
            interpretation = '🏘️ Maison haut de gamme'
        else:
            interpretation = '🏰 Maison de luxe'
        
        return result, interpretation
        
    except Exception as e:
        import traceback
        return f'Erreur: {str(e)}', traceback.format_exc()

print('✅ Fonction de prédiction ML avec transformation log1p chargée!')

✅ Fonction de prédiction ML avec transformation log1p chargée!


---
## 3. Interface Gradio

In [5]:
# Créer l'interface Gradio
with gr.Blocks(title="🏠 Ames Housing Price Predictor", theme=gr.themes.Soft()) as demo:
    
    gr.Markdown("""
    # 🏠 Prédicteur de Prix Immobiliers - Ames Housing
    
    Entrez les caractéristiques de la maison pour obtenir une estimation du prix.
    
    **Modèle utilisé:** Random Forest / Gradient Boosting  
    **Performance:** R² > 0.85, RMSE < $25,000
    """)
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📊 Qualité et Taille")
            overall_qual = gr.Slider(1, 10, value=6, step=1, label="Qualité Générale (1-10)")
            gr_liv_area = gr.Slider(500, 5000, value=1500, step=50, label="Surface Habitable (sq ft)")
            total_bsmt_sf = gr.Slider(0, 3000, value=1000, step=50, label="Surface Sous-sol (sq ft)")
            first_flr_sf = gr.Slider(500, 3000, value=1000, step=50, label="Surface 1er Étage (sq ft)")
            
        with gr.Column():
            gr.Markdown("### 🚗 Garage et Équipements")
            garage_cars = gr.Slider(0, 4, value=2, step=1, label="Capacité Garage (voitures)")
            garage_area = gr.Slider(0, 1000, value=500, step=50, label="Surface Garage (sq ft)")
            year_built = gr.Slider(1900, 2010, value=1990, step=1, label="Année de Construction")
            
        with gr.Column():
            gr.Markdown("### 🛁 Intérieur")
            full_bath = gr.Slider(0, 4, value=2, step=1, label="Salles de Bain Complètes")
            totrms_abvgrd = gr.Slider(2, 12, value=6, step=1, label="Nombre de Pièces")
            fireplaces = gr.Slider(0, 4, value=1, step=1, label="Cheminées")
    
    with gr.Row():
        predict_btn = gr.Button("🔮 Prédire le Prix", variant="primary", size="lg")
    
    with gr.Row():
        output_price = gr.Textbox(label="💰 Prix Estimé", lines=2)
        output_category = gr.Textbox(label="📊 Catégorie", lines=2)
    
    predict_btn.click(
        fn=predict_price,
        inputs=[overall_qual, gr_liv_area, garage_cars, garage_area, 
                total_bsmt_sf, first_flr_sf, year_built, full_bath,
                totrms_abvgrd, fireplaces],
        outputs=[output_price, output_category]
    )
    
    gr.Markdown("""
    ---
    ### ℹ️ À propos
    
    Cette application utilise un modèle de Machine Learning entraîné sur le dataset Ames Housing.
    
    **Variables les plus importantes:**
    - Qualité Générale (OverallQual)
    - Surface Habitable (GrLivArea)
    - Surface du Garage
    - Année de Construction
    
    **Disclaimer:** Les prédictions sont des estimations basées sur des données historiques.
    """)

In [ ]:
# Lancer l'application
print("🚀 Lancement de l'application Gradio...")
print("📌 L'interface s'ouvrira dans votre navigateur.")
print("📌 Pour arrêter, utilisez Ctrl+C ou redémarrez le kernel.")

demo.launch(share=False, inbrowser=True)